# Advent of code 2024
## Challenge 15

For this challenge, I only took help from ChatGPT to find the proper sorting function to sort a list of list according to the first element of each sublist.

## Part 1
### https://adventofcode.com/2024/day/15

In [1]:
input_file = open("challenge_15_input.txt", "r")
maze = []
directions = ""
start_position = [0,0]
direction_dictionary = {'^': [-1,0], 'v': [1,0], '>': [0,1], '<': [0,-1]}
opposite_direction_dictionary = {'^': [1,0], 'v': [-1,0], '>': [0,-1], '<': [0,1]}
stop_position = []
coordinates_sum = 0

# We first read the maze, and stops at the first empty line
for line in input_file:
    if line == "\n":
        break
    input_data = line.strip()
    maze.append(list(input_data))

# We then read the movements and load everything into a single string
# that we then transform into a list
for line in input_file:
    directions += line.strip()
directions = list(directions)

# We find the starting position
for main_index in range(len(maze)):
    for sub_index in range(len(maze[0])):
        if maze[main_index][sub_index] == '@':
            start_position = [main_index,sub_index]
            break

# The stop pisition is used to find the position at which the next action will be taken, which
# may be passed a stack of blocks
stop_position = start_position.copy()
 
for direction in directions:
    # This while loop finds the first position that is either an empty space or an edge. It goes through
    # the blocks
    while True:
        stop_position[0] += direction_dictionary[direction][0]
        stop_position[1] += direction_dictionary[direction][1]
        if (maze[stop_position[0]][stop_position[1]] == '#' or maze[stop_position[0]][stop_position[1]] == '.'):
            break

    # If the space at which we stopped above is an empty space, then there can be a movement the of robot
    if maze[stop_position[0]][stop_position[1]] == '.':
        # in this while loop, we move both the robots and all the blocks he may be pushing. We move everything
        # one step ahead in the current direction
        # After moveing the robot, the move is over and so we break from the while loop
        while True:
            if maze[stop_position[0] + opposite_direction_dictionary[direction][0]][stop_position[1] + opposite_direction_dictionary[direction][1]] == '@':
                maze[stop_position[0] + opposite_direction_dictionary[direction][0]][stop_position[1] + opposite_direction_dictionary[direction][1]] = '.'
                maze[stop_position[0]][stop_position[1]] = '@'
                break

            maze[stop_position[0]][stop_position[1]] = 'O'
            stop_position[0] += opposite_direction_dictionary[direction][0]
            stop_position[1] += opposite_direction_dictionary[direction][1]
        # We reset the stop position at the start position for the next move
        start_position = stop_position.copy()
    else:
        # If the position passed the blocks was an edge, nothing is done and the stop position is reset back to the 
        # start position for the next move
        stop_position = start_position.copy()

# We calculate the coordinates of each block position and add it all together to get the answer
for main_index in range(len(maze)):
    for sub_index in range(len(maze[0])):
        if maze[main_index][sub_index] == 'O':
            coordinates_sum += (main_index * 100) + sub_index
    
print(coordinates_sum)
for line in maze:
    print("".join(line))    

1383666
##################################################
#O...OOOOOO.#OO.....#O..#O.......#.OO..OO.....OOO#
#O..#....#.#O.O#O...OOO.......O..OOOOOOO##......O#
#.....#...OO....OO....O.....#....OOO..OO.........#
#..........O.......OO.OO....##.O...OO..O.....O...#
#OO.#.#...#O#...O.....O#.....O...O.....O.O#.....O#
#OO#OO.....OOOO#.................O....OO...O....O#
#.OOO#.....O##....OO.....#OOO..........OO..O..#.O#
#.O.OO..........O#OO....OOO....OO...OO.O...OO...##
#..#.#O......#....OO...OOO......#.O.....OO.O.#.OO#
#O...OOOO...OO.......OOOOO#.......#......O..O..O.#
#O....O...#.#O......O.O.O.O..#..#.....#.....##.OO#
#O..#.......O...........OOO....#O....O#...O..O.O.#
#.......................#.....#O...#......O..O.O.#
#......#.....O#.#O.OO.......OOO............O.O.O.#
#.........#..O.OOOO#..O......OO.............#O.O.#
##O..#.O#.O....O..OOO...........OOO.O..........O.#
#OO....#O...#.......O...OOO.....O.#..OO........O##
#OO....OO........OOOO#....#...O......##.....#....#
#OO.......#......O#OOO.

## Part 2

In [2]:
# This function gathers all of the positions that form a block vertically. 
# This function was needed because blocks can form pyramids, then the whole pyramid has to be collected
def find_block(position, maze, direction, is_blocked):
    # We add the position to the blocks list at the start, only if it is not already in the list
    if position not in found_blocks:
        found_blocks.append(position)
    
    # We recurse on the opposite side of a block that was cought
    if maze[position[0]][position[1]] == '[' and [position[0],position[1] + 1] not in found_blocks:
        find_block([position[0],position[1] + 1], maze, direction, is_blocked)
    elif maze[position[0]][position[1]] == ']' and [position[0],position[1] - 1] not in found_blocks:
        find_block([position[0],position[1] - 1], maze, direction, is_blocked)
    
    # This is the pyramid part, if there is a part of a block under another part of a block, we recurse on that part
    if (maze[position[0] + direction[0]][position[1] + direction[1]] == '[' or maze[position[0] + direction[0]][position[1] + direction[1]] == ']') and maze[position[0] + direction[0]][position[1] + direction[1]] not in found_blocks:
        find_block([position[0] + direction[0],position[1] + direction[1]], maze, direction, is_blocked)
    # If any one of the blocks is touching an edge, there will be no move
    elif maze[position[0] + direction[0]][position[1] + direction[1]] == '#':
        is_blocked[0] = True

In [3]:
# Reading the data input file
input_file = open("challenge_15_input.txt", "r")
maze = []
maze_line = []
directions = ""
start_position = [0,0]
direction_dictionary = {'^': [-1,0], 'v': [1,0], '>': [0,1], '<': [0,-1]}
opposite_direction_dictionary = {'^': [1,0], 'v': [-1,0], '>': [0,-1], '<': [0,1]}
stop_position = []
coordinates_sum = 0
found_blocks = []
# A list is used because the changes made in a recursive function are saved.
# If it would only be a boolean, the change made in the recursion would be lost
is_blocked = [False]

# We create the large maze according to specifications
for line in input_file:
    if line == "\n":
        break
    input_data = line.strip()
    input_data = list(input_data)
    for data_point in input_data:
        if data_point == '#':
            maze_line.append('#')
            maze_line.append('#')
        elif data_point == 'O':
            maze_line.append('[')
            maze_line.append(']')
        elif data_point == '.':
            maze_line.append('.')
            maze_line.append('.')
        elif data_point == '@':
            maze_line.append('@')
            maze_line.append('.')
            
    maze.append(maze_line)
    maze_line = []

for line in input_file:
    directions += line.strip()
    
directions = list(directions)

for main_index in range(len(maze)):
    for sub_index in range(len(maze[0])):
        if maze[main_index][sub_index] == '@':
            start_position = [main_index,sub_index]
            break

stop_position = start_position.copy()
    
for direction in directions:
    # If the step ahead is an empty space, we move the robot
    if (maze[start_position[0] + direction_dictionary[direction][0]][start_position[1] + direction_dictionary[direction][1]] == '.'):
        maze[start_position[0]][start_position[1]] = '.'
        start_position[0] += direction_dictionary[direction][0]
        start_position[1] += direction_dictionary[direction][1]
        maze[start_position[0]][start_position[1]] = '@'
        stop_position = start_position.copy()
    
    # If there are blocks and the direction is horizontal, we follow the same method as in part 1 of the challenge
    elif ((direction == '>' or direction == '<') and (maze[start_position[0] + direction_dictionary[direction][0]][start_position[1] + direction_dictionary[direction][1]] == '[' or maze[start_position[0] + direction_dictionary[direction][0]][start_position[1] + direction_dictionary[direction][1]] == ']')):
        while True:
            stop_position[0] += direction_dictionary[direction][0]
            stop_position[1] += direction_dictionary[direction][1]
            if (maze[stop_position[0]][stop_position[1]] == '#' or maze[stop_position[0]][stop_position[1]] == '.'):
                break

        if maze[stop_position[0]][stop_position[1]] == '.':
            while True:
                if maze[stop_position[0] + opposite_direction_dictionary[direction][0]][stop_position[1] + opposite_direction_dictionary[direction][1]] == '@':
                    maze[stop_position[0] + opposite_direction_dictionary[direction][0]][stop_position[1] + opposite_direction_dictionary[direction][1]] = '.'
                    maze[stop_position[0]][stop_position[1]] = '@'
                    break

                maze[stop_position[0]][stop_position[1]] = maze[stop_position[0] + opposite_direction_dictionary[direction][0]][stop_position[1] + opposite_direction_dictionary[direction][1]]
                stop_position[0] += opposite_direction_dictionary[direction][0]
                stop_position[1] += opposite_direction_dictionary[direction][1]
            start_position = stop_position.copy()
        else:
            stop_position = start_position.copy()
    
    # If the direction is vertical, we use the recursive function to get the potential pyramid of blocks
    elif ((direction == 'v' or direction == '^') and (maze[start_position[0] + direction_dictionary[direction][0]][start_position[1] + direction_dictionary[direction][1]] == '[' or maze[start_position[0] + direction_dictionary[direction][0]][start_position[1] + direction_dictionary[direction][1]] == ']')):
        # We append right away in the list of found blocks the position of the robot, because he too will be moved
        found_blocks.append(start_position.copy())
        
        # We use the recursive function to find all the blocks the robot would be pushing
        find_block([start_position[0] + direction_dictionary[direction][0],start_position[1] + direction_dictionary[direction][1]], maze, direction_dictionary[direction], is_blocked)
        
        # If the set of blocks touches at least on piece of edge or wall, the move doesn't happen
        # The parameters are reset and we move right away to the next iteration
        if is_blocked[0]:
            found_blocks = []
            is_blocked = [False]
            continue
        
        # If the block can be moved, we first sort the blocks
        # If the direction is up, we sort in ascending order, to move pieces of the top rows first
        if direction == '^':
            found_blocks = sorted(found_blocks, key=lambda x: x[0], reverse=False)
        else:
        # If the direction is down, we sort descending, to move the pieces at the bottom rows first
        # If we don't do that, pices will be lost because pieces that has to be moved will have been
        # overwritten by some other pieces
            found_blocks = sorted(found_blocks, key=lambda x: x[0], reverse=True)
        
        # Then we do the movement. When a row is moved, the previous position is marked as empty, until 
        # the pieces come on it. This is done so that the last pieces of a column that are moved have an
        # empty space mark on their previous space. Because those will not have any new pieces coming on them
        for block in found_blocks:
            maze[block[0] + direction_dictionary[direction][0]][block[1] + direction_dictionary[direction][1]] = maze[block[0]][block[1]]
            maze[block[0]][block[1]] = '.'
        
        # we reset our variables
        found_blocks = []
        is_blocked = [False]
        # The start position is moved to be the same as the robot
        start_position[0] += direction_dictionary[direction][0]
        start_position[1] += direction_dictionary[direction][1]
        # the stop position is put back at the start position
        stop_position = start_position.copy()
 
# We calculate the sum of the coordinates according to specifications
for main_index in range(len(maze)):
    for sub_index in range(len(maze[0])):
        if maze[main_index][sub_index] == '[':
            coordinates_sum += (main_index * 100) + sub_index

# We print the maze
print(coordinates_sum)
for line in maze:
    print("".join(line))

1412866
####################################################################################################
##......[]........[][]..##[]........[][]##..[]..##.....[][].[]..[]##[]..[].[].[][]..............[]##
##..[][]##........##..##[]....##.......[]..[][][].......[]....[][].....[]....[].####....[][]......##
##[]........##[]................................[]......##.....[]..............[].............[]..##
##................[]...........................[][].....####.[][]...............[]..[][]..[]..[]..##
##......##..##......##..##[].........[]..[]...##[]....[][]......................[][]##....[]..[]..##
##..[]##[][][]..[]..[]....[][]##[]..[]..[].....[][][]..........[]...............[]......[]....[][]##
##....[][]##[]..[][]....####[][][]................##..............[][]..............[]......##[][]##
##.......[].......[][]......[][][]##...............................[].................[][]....[]####
##..[]##..##..........[]..##[][][]...................[].........##..[][]..........[